In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("result_seq2seq.csv")

In [4]:
data["answer"] = data["answer"].str.replace('_', '')

data

In [7]:
ans = data["answer"]
pred = data["predict"]

In [9]:
ans

0                        まー \r\n
1         うん 運動 好き だっ た ん だ \r\n
2               あー あはは っ はは \r\n
3                      ほうほう \r\n
4                   そう です ね \r\n
                  ...           
22195                    ああ \r\n
22196                    あー \r\n
22197                ビー玉 はい \r\n
22198                    ああ \r\n
22199    え え スキー を やら れ て て \r\n
Name: answer, Length: 22200, dtype: object

# 完全一致率の計算

In [10]:
cnt = 0
for i in range(len(ans)):
    if ans[i] == pred[i]:
        #print("ans:"+ans[i]+" pred:"+pred[i])
        cnt+=1

In [11]:
cnt

1262

In [12]:
same_percentage = (cnt/len(ans))*100

In [13]:
same_percentage

5.684684684684685

# 種類数の計算

In [14]:
col = []
for i in range(len(pred)):
    if pred[i] not in col:
        #print(pred[i])
        col.append(pred[i])

In [15]:
len(col)

49

# BLUEの計算

入力に対する答えが複数あるのですべてに対してスコアを計算する


In [16]:
import nltk

In [17]:
nltk.translate.bleu_score.sentence_bleu(ans,pred)

0

In [18]:
ans

0                        まー \r\n
1         うん 運動 好き だっ た ん だ \r\n
2               あー あはは っ はは \r\n
3                      ほうほう \r\n
4                   そう です ね \r\n
                  ...           
22195                    ああ \r\n
22196                    あー \r\n
22197                ビー玉 はい \r\n
22198                    ああ \r\n
22199    え え スキー を やら れ て て \r\n
Name: answer, Length: 22200, dtype: object